# STEM CTF MITRE 2019

### 59 - Binary 50

The solution is simple, just run strings on the binary and grep for the stat of the flag.

```bash
$ strings challenge | grep MCA
MCA{y3ah_sur3_here_y0u_g0}
```

### 61 - Grab bag 100
The original image has blown out background, either too black or too white. Guessing this meant some data whas hidden there as the image was viewable, as is.

Found a threshold function in convert and ran it for white, and the first part appeared.  Figured out how to run the black threshold at the website below, and tried that as well. 
https://imagemagick.org/script/command-line-options.php#black-threshold


`convert Scaredy_Cat.png -white-threshold 0% white.png`  
`convert Scaredy_Cat.png -black-threshold 100% black.png`


![Alt text](./solution_61_white.png "a title")
![Alt text](./solution_61_black.png "a title")


### 62 - Grab bag 100
A recursive loop of decompressing a archive ~500 times, based on archive type, in order to get the flag.

In [ ]:
#!/bin/bash
#URL FOR file signatures https://www.garykessler.net/library/file_sigs.html
FNAME=flag
TRUE=1
C=0
ZERO="000"

while [[ $TRUE -lt 502 ]]; do
	if [[ $C -lt 10 ]]; then
		ZERO="000"
	elif [[ $C -lt 100 ]]; then
		ZERO="00"
	elif [[ $C -lt 1000 ]]; then
		ZERO="0"
	else
		ZERO=""
	fi

	##Grab filesignature
	FHEAD=$(hd "$FNAME" | head -n1 | awk -F" " '{print $2$3$4}')

	echo -n "$FHEAD		$FNAME		"

	if [[ $FHEAD == "1f8b08" ]]; then
		#GZ
		gzip --decompress "$FNAME" --stdout > "flag_$ZERO$C"
		echo "gzip"
	elif [[ $FHEAD == "425a68" ]]; then
		#BZ2
		bzip2 --decompress "$FNAME" --stdout > "flag_$ZERO$C"
		echo "bz2"
	elif [[ $FHEAD == "504b03" ]]; then
		#ZIP 
		gunzip -c "$FNAME" > "flag_$ZERO$C"
		echo "ZIP"
	else
		#base64
		cat "$FNAME" | base64 --decode > "flag_$ZERO$C"
		echo "b64"
	fi
	
	FNAME="flag_$ZERO$C"
	
	((C=$C+1))
	((TRUE=$TRUE+1))
done

### 77 - Web 50
- Accessing the site gave Certificate error request.
- Granting a permission did nothing as the page eventually reloaded itself.
- Checking the certificate gave the "Certificate Subject Alt Name"  
    "DNS Name: 138.247.13.115.xip.io"
- Trying the dns name instead also gave nothing, but a unable to connect error
- Putting it in HTTPS, as it has a certificate grants us the flag  
    **Flag:** `MCA{shuHeimoowaiF5a}`

![Alt text](./solution_77_web_50.png "a title")

### 81 - Linux 50

The task is to break out of an RBASH and find the flag, and trying standard commands resulted in failures where nothing was allowed.

After looking for information I found the following presentation.  
https://speakerdeck.com/knaps/escape-from-shellcatraz-breaking-out-of-restricted-unix-shells?slide=9

Using it I broke out after trying Step 4, by issuing a command prior to establishing a shell  
`ssh ctf@138.247.13.108 -t "bash --noprofile"`

Then finding the flag was harder as the server does not have locate, but it was located in plaintext in the root directory.  
```
jollyjackson@jollyjackson-laptop:~/Downloads$ ssh ctf@138.247.13.108 -t "bash --noprofile"
ctf@502b04eba365:~$ cd /root/
ctf@502b04eba365:/root$ cat flag.txt 
MCA{ieHaisoh4eif2ae}
ctf@502b04eba365:/root$ 
```